In [190]:
import argparse
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

class Layer():
    def __init__(self, input_dim, output_dim, idx):
        self.layer = nn.Linear(input_dim, output_dim)
        self.activations = None
        self.idx = idx

    def forward(self, x):
        self.activations = self.layer(x)
        return self.activations
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = Layer(320, 150, 0)
        self.fc2 = Layer(150, 150, 1)
        self.fc3 = Layer(150, 150, 2)
        self.fc4 = Layer(150, 150, 3)
        self.fc5 = Layer(150, 10, 4)
        
        self.prunable_layers = [self.fc1,self.fc2,self.fc3,self.fc4,self.fc5] 

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1.forward(x))
        x = F.relu(self.fc2.forward(x))
        x = F.relu(self.fc3.forward(x))
        x = F.relu(self.fc4.forward(x))
        x = self.fc5.forward(x)
        return F.log_softmax(x, dim=1)
    
    def remove_nodes(self, to_remove):
        for layer_index, nodes_to_remove in to_remove.items():
            n_remove = len(to_remove)
            layer = self.prunable_layers[layer_index]
            layer.layer.out_features -= n_remove
            
            # delete layer_index row in layer, and column in next layer
            np_weights = layer.layer.weight.detach().numpy()
            np_weights = np.delete(np_weights, nodes_to_remove, axis=0)
            layer.layer.weight = Parameter(torch.Tensor(np_weights))
            if(layer.layer.bias is not None):
                layer_weights = layer.layer.bias.detach().numpy()
                layer_weights = np.delete(layer_weights, nodes_to_remove)
                layer.layer.bias = Parameter(torch.Tensor(layer_weights))
            
            next_layer = self.prunable_layers[layer_index + 1]
            next_layer.layer.in_features -= 1
            np_weights = next_layer.layer.weight.detach().numpy()
            np_weights = np.delete(np_weights, nodes_to_remove, axis=1)
            next_layer.layer.weight = Parameter(torch.Tensor(np_weights))

        '''assert layer_index < 4
        layer = self.prunable_layers[layer_index]
        assert(node < self.prunable_layers[layer_index].layer.out_features )
        layer.layer.out_features -= 1
        # delete layer_index row in layer, and column in next layer
        np_weights = layer.layer.weight.detach().numpy()
        np_weights = np.delete(np_weights, layer_index, axis=0)
        layer.layer.weight = Parameter(torch.Tensor(np_weights))
        if(layer.layer.bias is not None):
            layer_weights = layer.layer.bias.detach().numpy()
            layer_weights = np.delete(layer_weights, layer_index)
            layer.layer.bias = Parameter(torch.Tensor(layer_weights))
        next_layer = self.prunable_layers[layer_index + 1]
        next_layer.layer.in_features -= 1
        np_weights = next_layer.layer.weight.detach().numpy()
        np_weights = np.delete(np_weights, layer_index, axis=1)
        next_layer.layer.weight = Parameter(torch.Tensor(np_weights))
        
        print(layer.layer.weight.shape)
        print(next_layer.layer.weight.shape)'''

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [191]:
torch.manual_seed(1)

device = torch.device("cpu")

kwargs = {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [192]:
for epoch in range(10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

val_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=10000, shuffle=True, **kwargs)
test(model, device, val_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.293017
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.296916
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.320870
Train Epoch: 0 [1920/60000 (3%)]	Loss: 2.305141
Train Epoch: 0 [2560/60000 (4%)]	Loss: 2.306541
Train Epoch: 0 [3200/60000 (5%)]	Loss: 2.302220
Train Epoch: 0 [3840/60000 (6%)]	Loss: 2.296724
Train Epoch: 0 [4480/60000 (7%)]	Loss: 2.297585
Train Epoch: 0 [5120/60000 (9%)]	Loss: 2.306103
Train Epoch: 0 [5760/60000 (10%)]	Loss: 2.304100
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.297133
Train Epoch: 0 [7040/60000 (12%)]	Loss: 2.303020
Train Epoch: 0 [7680/60000 (13%)]	Loss: 2.312071
Train Epoch: 0 [8320/60000 (14%)]	Loss: 2.300865
Train Epoch: 0 [8960/60000 (15%)]	Loss: 2.301143
Train Epoch: 0 [9600/60000 (16%)]	Loss: 2.314250
Train Epoch: 0 [10240/60000 (17%)]	Loss: 2.302002
Train Epoch: 0 [10880/60000 (18%)]	Loss: 2.300359
Train Epoch: 0 [11520/60000 (19%)]	Loss: 2.301488
Train Epoch: 0 [12160/60000 (20%)]	Loss: 2.307114
Train Epoch: 0 [12800/60000 (

Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.281880
Train Epoch: 1 [45440/60000 (76%)]	Loss: 2.306848
Train Epoch: 1 [46080/60000 (77%)]	Loss: 2.307113
Train Epoch: 1 [46720/60000 (78%)]	Loss: 2.303772
Train Epoch: 1 [47360/60000 (79%)]	Loss: 2.315668
Train Epoch: 1 [48000/60000 (80%)]	Loss: 2.301436
Train Epoch: 1 [48640/60000 (81%)]	Loss: 2.299709
Train Epoch: 1 [49280/60000 (82%)]	Loss: 2.305537
Train Epoch: 1 [49920/60000 (83%)]	Loss: 2.311293
Train Epoch: 1 [50560/60000 (84%)]	Loss: 2.301433
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.298042
Train Epoch: 1 [51840/60000 (86%)]	Loss: 2.298600
Train Epoch: 1 [52480/60000 (87%)]	Loss: 2.313994
Train Epoch: 1 [53120/60000 (88%)]	Loss: 2.289097
Train Epoch: 1 [53760/60000 (90%)]	Loss: 2.288143
Train Epoch: 1 [54400/60000 (91%)]	Loss: 2.298118
Train Epoch: 1 [55040/60000 (92%)]	Loss: 2.292164
Train Epoch: 1 [55680/60000 (93%)]	Loss: 2.305789
Train Epoch: 1 [56320/60000 (94%)]	Loss: 2.297435
Train Epoch: 1 [56960/60000 (95%)]	Loss: 2.308891


Train Epoch: 3 [28800/60000 (48%)]	Loss: 2.311592
Train Epoch: 3 [29440/60000 (49%)]	Loss: 2.282160
Train Epoch: 3 [30080/60000 (50%)]	Loss: 2.282972
Train Epoch: 3 [30720/60000 (51%)]	Loss: 2.281662
Train Epoch: 3 [31360/60000 (52%)]	Loss: 2.298681
Train Epoch: 3 [32000/60000 (53%)]	Loss: 2.287302
Train Epoch: 3 [32640/60000 (54%)]	Loss: 2.292421
Train Epoch: 3 [33280/60000 (55%)]	Loss: 2.289165
Train Epoch: 3 [33920/60000 (57%)]	Loss: 2.277255
Train Epoch: 3 [34560/60000 (58%)]	Loss: 2.297353
Train Epoch: 3 [35200/60000 (59%)]	Loss: 2.281937
Train Epoch: 3 [35840/60000 (60%)]	Loss: 2.288062
Train Epoch: 3 [36480/60000 (61%)]	Loss: 2.296213
Train Epoch: 3 [37120/60000 (62%)]	Loss: 2.297399
Train Epoch: 3 [37760/60000 (63%)]	Loss: 2.268242
Train Epoch: 3 [38400/60000 (64%)]	Loss: 2.284864
Train Epoch: 3 [39040/60000 (65%)]	Loss: 2.272147
Train Epoch: 3 [39680/60000 (66%)]	Loss: 2.278936
Train Epoch: 3 [40320/60000 (67%)]	Loss: 2.299782
Train Epoch: 3 [40960/60000 (68%)]	Loss: 2.281452


Train Epoch: 5 [12800/60000 (21%)]	Loss: 2.259949
Train Epoch: 5 [13440/60000 (22%)]	Loss: 2.310836
Train Epoch: 5 [14080/60000 (23%)]	Loss: 2.245173
Train Epoch: 5 [14720/60000 (25%)]	Loss: 2.299227
Train Epoch: 5 [15360/60000 (26%)]	Loss: 2.260664
Train Epoch: 5 [16000/60000 (27%)]	Loss: 2.323499
Train Epoch: 5 [16640/60000 (28%)]	Loss: 2.262493
Train Epoch: 5 [17280/60000 (29%)]	Loss: 2.262259
Train Epoch: 5 [17920/60000 (30%)]	Loss: 2.250179
Train Epoch: 5 [18560/60000 (31%)]	Loss: 2.265314
Train Epoch: 5 [19200/60000 (32%)]	Loss: 2.240224
Train Epoch: 5 [19840/60000 (33%)]	Loss: 2.273004
Train Epoch: 5 [20480/60000 (34%)]	Loss: 2.212778
Train Epoch: 5 [21120/60000 (35%)]	Loss: 2.276608
Train Epoch: 5 [21760/60000 (36%)]	Loss: 2.267460
Train Epoch: 5 [22400/60000 (37%)]	Loss: 2.278491
Train Epoch: 5 [23040/60000 (38%)]	Loss: 2.256346
Train Epoch: 5 [23680/60000 (39%)]	Loss: 2.265061
Train Epoch: 5 [24320/60000 (41%)]	Loss: 2.276249
Train Epoch: 5 [24960/60000 (42%)]	Loss: 2.255018


Train Epoch: 6 [57600/60000 (96%)]	Loss: 2.230697
Train Epoch: 6 [58240/60000 (97%)]	Loss: 2.230223
Train Epoch: 6 [58880/60000 (98%)]	Loss: 2.233250
Train Epoch: 6 [59520/60000 (99%)]	Loss: 2.218353

Test set: Average loss: 2.2139, Accuracy: 2576/10000 (26%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 2.196948
Train Epoch: 7 [640/60000 (1%)]	Loss: 2.261846
Train Epoch: 7 [1280/60000 (2%)]	Loss: 2.243233
Train Epoch: 7 [1920/60000 (3%)]	Loss: 2.189997
Train Epoch: 7 [2560/60000 (4%)]	Loss: 2.228001
Train Epoch: 7 [3200/60000 (5%)]	Loss: 2.226238
Train Epoch: 7 [3840/60000 (6%)]	Loss: 2.206750
Train Epoch: 7 [4480/60000 (7%)]	Loss: 2.234880
Train Epoch: 7 [5120/60000 (9%)]	Loss: 2.292190
Train Epoch: 7 [5760/60000 (10%)]	Loss: 2.264921
Train Epoch: 7 [6400/60000 (11%)]	Loss: 2.250982
Train Epoch: 7 [7040/60000 (12%)]	Loss: 2.197752
Train Epoch: 7 [7680/60000 (13%)]	Loss: 2.297733
Train Epoch: 7 [8320/60000 (14%)]	Loss: 2.252678
Train Epoch: 7 [8960/60000 (15%)]	Loss: 2.235269
Train Epoch: 7 [9

Train Epoch: 8 [41600/60000 (69%)]	Loss: 2.188895
Train Epoch: 8 [42240/60000 (70%)]	Loss: 2.219390
Train Epoch: 8 [42880/60000 (71%)]	Loss: 2.265861
Train Epoch: 8 [43520/60000 (72%)]	Loss: 2.215975
Train Epoch: 8 [44160/60000 (74%)]	Loss: 2.196788
Train Epoch: 8 [44800/60000 (75%)]	Loss: 2.201462
Train Epoch: 8 [45440/60000 (76%)]	Loss: 2.192506
Train Epoch: 8 [46080/60000 (77%)]	Loss: 2.260876
Train Epoch: 8 [46720/60000 (78%)]	Loss: 2.166515
Train Epoch: 8 [47360/60000 (79%)]	Loss: 2.201965
Train Epoch: 8 [48000/60000 (80%)]	Loss: 2.156469
Train Epoch: 8 [48640/60000 (81%)]	Loss: 2.244986
Train Epoch: 8 [49280/60000 (82%)]	Loss: 2.169785
Train Epoch: 8 [49920/60000 (83%)]	Loss: 2.210950
Train Epoch: 8 [50560/60000 (84%)]	Loss: 2.209990
Train Epoch: 8 [51200/60000 (85%)]	Loss: 2.212447
Train Epoch: 8 [51840/60000 (86%)]	Loss: 2.244809
Train Epoch: 8 [52480/60000 (87%)]	Loss: 2.163993
Train Epoch: 8 [53120/60000 (88%)]	Loss: 2.229436
Train Epoch: 8 [53760/60000 (90%)]	Loss: 2.133880


In [195]:
activ_list = []
n_layers = len(model.prunable_layers) - 1
for layer_num in range(n_layers - 1):
    layer = model.prunable_layers[layer_num]
    activs = np.var(layer.activations.detach().numpy(), axis=0)
    activ_list.append(activs)
    
max_nodes = max(len(row) for row in activ_list)
activations = np.empty((n_layers, max_nodes))
for layer_num, activs in enumerate(activ_list):
    activations[layer_num,:len(activs)] += activs

print(activations.ravel())
# https://stackoverflow.com/questions/30577375/have-numpy-argsort-return-an-array-of-2d-indices
sorted = np.dstack(np.unravel_index(np.argsort(activations.ravel()), activations.shape))[0]
indices_to_remove = sorted[:50, :]
to_remove = {}
print(indices_to_remove, sorted)
for i in indices_to_remove:
    layer_num = i[0]
    node_num = i[1]
    if layer_num in to_remove:
        to_remove[layer_num].append(node_num)
    else:
        to_remove[layer_num] = [node_num]
model.remove_nodes(to_remove)

[2.71762665e+002 1.48338211e+002 1.09639091e+002 1.62244736e+002
 1.59837540e+002 1.48485992e+002 2.20620159e+161 1.96266954e+243
 2.07958837e+262 2.14435303e+002 9.85330622e+165 6.67105512e+135
 1.11566383e+002 1.01883614e+002 1.07956940e+002 1.02436569e+002
 1.04195221e+002 1.14269356e+002 7.74549484e+001 1.16329964e+002
 1.76577759e+002 3.54014587e+002 2.11535327e+257 1.46971008e+002
 9.89803615e+164 8.89115793e+179 1.83573525e+223 2.37697104e+137
 9.02193423e+217 9.10611877e+001 2.03292755e+002 6.65878601e+001
 1.29315989e+161 8.90567240e+252 1.78410919e+002 6.09079069e+247
 1.52447601e+002 1.34668121e+002 4.63461863e+228 1.98453918e+002
 2.44475250e+002 1.59993698e+002 1.61410559e+132 2.11161559e+257
 2.83837555e+002 2.88067937e+214 9.89803615e+164 1.86718964e+002
 8.34413757e+001 1.62074188e+002 1.99692490e+002 2.11680463e+257
 2.05326996e+002 1.58298477e+002 1.00385544e+218 1.73470917e+002
 2.13412571e+257 1.15026436e+002 1.59346725e+002 1.36068726e+002
 4.82594598e+276 9.898036

/Users/rohitbandaru/Desktop/PruneNN/env/lib/python2.7/site-packages/ipykernel_launcher.py:53: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
/Users/rohitbandaru/Desktop/PruneNN/env/lib/python2.7/site-packages/ipykernel_launcher.py:57: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
/Users/rohitbandaru/Desktop/PruneNN/env/lib/python2.7/site-packages/ipykernel_launcher.py:63: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.


In [196]:
for layer in model.prunable_layers:
    print(layer.layer.weight.shape)
for epoch in range(5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

val_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=10000, shuffle=True, **kwargs)
test(model, device, val_loader)

torch.Size([150, 320])
torch.Size([150, 150])
torch.Size([150, 150])
torch.Size([65, 150])
torch.Size([10, 65])
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.216283
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.185538
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.255602
Train Epoch: 0 [1920/60000 (3%)]	Loss: 2.184619
Train Epoch: 0 [2560/60000 (4%)]	Loss: 2.245117
Train Epoch: 0 [3200/60000 (5%)]	Loss: 2.166049
Train Epoch: 0 [3840/60000 (6%)]	Loss: 2.260930
Train Epoch: 0 [4480/60000 (7%)]	Loss: 2.201599
Train Epoch: 0 [5120/60000 (9%)]	Loss: 2.297812
Train Epoch: 0 [5760/60000 (10%)]	Loss: 2.257277
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.163260
Train Epoch: 0 [7040/60000 (12%)]	Loss: 2.243027
Train Epoch: 0 [7680/60000 (13%)]	Loss: 2.212737
Train Epoch: 0 [8320/60000 (14%)]	Loss: 2.208912
Train Epoch: 0 [8960/60000 (15%)]	Loss: 2.262806
Train Epoch: 0 [9600/60000 (16%)]	Loss: 2.226207
Train Epoch: 0 [10240/60000 (17%)]	Loss: 2.190095
Train Epoch: 0 [10880/60000 (18%)]	Loss: 2.180599
Train Epoch: 0 [1

Train Epoch: 1 [43520/60000 (72%)]	Loss: 2.155515
Train Epoch: 1 [44160/60000 (74%)]	Loss: 2.224922
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.186510
Train Epoch: 1 [45440/60000 (76%)]	Loss: 2.185141
Train Epoch: 1 [46080/60000 (77%)]	Loss: 2.195756
Train Epoch: 1 [46720/60000 (78%)]	Loss: 2.272605
Train Epoch: 1 [47360/60000 (79%)]	Loss: 2.196863
Train Epoch: 1 [48000/60000 (80%)]	Loss: 2.253438
Train Epoch: 1 [48640/60000 (81%)]	Loss: 2.171443
Train Epoch: 1 [49280/60000 (82%)]	Loss: 2.186246
Train Epoch: 1 [49920/60000 (83%)]	Loss: 2.231385
Train Epoch: 1 [50560/60000 (84%)]	Loss: 2.201132
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.173633
Train Epoch: 1 [51840/60000 (86%)]	Loss: 2.250235
Train Epoch: 1 [52480/60000 (87%)]	Loss: 2.183075
Train Epoch: 1 [53120/60000 (88%)]	Loss: 2.196256
Train Epoch: 1 [53760/60000 (90%)]	Loss: 2.250664
Train Epoch: 1 [54400/60000 (91%)]	Loss: 2.177114
Train Epoch: 1 [55040/60000 (92%)]	Loss: 2.210745
Train Epoch: 1 [55680/60000 (93%)]	Loss: 2.220427


Train Epoch: 3 [27520/60000 (46%)]	Loss: 2.171176
Train Epoch: 3 [28160/60000 (47%)]	Loss: 2.203087
Train Epoch: 3 [28800/60000 (48%)]	Loss: 2.130406
Train Epoch: 3 [29440/60000 (49%)]	Loss: 2.144857
Train Epoch: 3 [30080/60000 (50%)]	Loss: 2.208975
Train Epoch: 3 [30720/60000 (51%)]	Loss: 2.184241
Train Epoch: 3 [31360/60000 (52%)]	Loss: 2.209623
Train Epoch: 3 [32000/60000 (53%)]	Loss: 2.156405
Train Epoch: 3 [32640/60000 (54%)]	Loss: 2.164232
Train Epoch: 3 [33280/60000 (55%)]	Loss: 2.144001
Train Epoch: 3 [33920/60000 (57%)]	Loss: 2.125215
Train Epoch: 3 [34560/60000 (58%)]	Loss: 2.153698
Train Epoch: 3 [35200/60000 (59%)]	Loss: 2.080014
Train Epoch: 3 [35840/60000 (60%)]	Loss: 2.165434
Train Epoch: 3 [36480/60000 (61%)]	Loss: 2.182506
Train Epoch: 3 [37120/60000 (62%)]	Loss: 2.152959
Train Epoch: 3 [37760/60000 (63%)]	Loss: 2.131560
Train Epoch: 3 [38400/60000 (64%)]	Loss: 2.170823
Train Epoch: 3 [39040/60000 (65%)]	Loss: 2.252130
Train Epoch: 3 [39680/60000 (66%)]	Loss: 2.169024
